In [33]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [34]:
train_image_directory = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train"
validation_image_directory = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation"

In [35]:
filename_train = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-train.csv"
df_train = pd.read_csv(filename_train, header = 0)
filename_test = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-test.csv"
df_test = pd.read_csv(filename_test, header = 0)
filename_val = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-validation.csv"
df_val = pd.read_csv(filename_val, header = 0)

In [36]:
df_train.head(10)

,uniqueID,classLabel,classID,source,imageFile
0,2,occluded-specimens,8,L,a1a8b48e8cb142b3.jpg
1,3,microscope-slides,6,L,79599db2ac9092b6.jpg
2,4,illustrations-color,2,BHL,c449696f2f0d0d92.jpg
3,5,illustrations-color,2,P,80a8f4a393b4e08c.jpg
4,6,animal-specimens,0,AK,041a1c6e73313638.jpg
5,8,occluded-specimens,8,L,ccf1b1ccacb8f8b3.jpg
6,11,live-plants,4,US,246c8e9612111a24.jpg
7,12,biocultural-specimens,1,C,b0b337313164a0f0.jpg
8,13,microscope-slides,6,L,b3931bc4cc2b2925.jpg
9,15,illustrations-gray,3,BR,ebf7674e4c2c0e6e.jpg


In [37]:
print(df_train.columns)
print(df_val.columns)
print(df_test.columns)

Index(['uniqueID', 'classLabel', 'classID', 'source', 'imageFile'], dtype='object')
Index(['uniqueID', 'classLabel', 'classID', 'source', 'imageFile'], dtype='object')
Index(['uniqueID', 'imageFile'], dtype='object')


In [38]:
# train a Small VGGNet using df_train, df_test, and df_val
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

df_train["classID"] = df_train["classID"].astype(str)
df_val["classID"] = df_val["classID"].astype(str)

# load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

In [39]:
# construct the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dense(32, activation="relu")(headModel)
headModel = Dense(16, activation="relu")(headModel)
headModel = Dense(8, activation="relu")(headModel)
headModel = Dense(4, activation="relu")(headModel)
headModel = Dense(2, activation="relu")(headModel)
headModel = Dense(10, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False


In [40]:
# import the legacy SGD optimizer
from tensorflow.keras.optimizers.legacy import SGD

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random

[INFO] compiling model...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [41]:
# train the model using the training data
print("[INFO] training head...")
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(dataframe=df_train, directory=train_image_directory, x_col="imageFile", y_col="classID", class_mode="sparse", target_size=(224, 224), batch_size=32)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(dataframe=df_val, directory=validation_image_directory, x_col="imageFile", y_col="classID", class_mode="sparse", target_size=(224, 224), batch_size=32)


[INFO] training head...
Found 81945 validated image filenames belonging to 10 classes.
Found 10244 validated image filenames belonging to 10 classes.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="imageFile". These filename(s) will be ignored.
  warnings.warn(


## THIS ONE TAKES A WHILE TO RUN

In [42]:
model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10
2561/2561 [==============================] - 5425s 2s/step - loss: 2.1858 - accuracy: 0.1743 - val_loss: 2.0752 - val_accuracy: 0.1920
Epoch 2/10
2561/2561 [==============================] - 5402s 2s/step - loss: 2.0286 - accuracy: 0.1903 - val_loss: 1.9645 - val_accuracy: 0.1976
Epoch 3/10
2561/2561 [==============================] - 5412s 2s/step - loss: 1.9512 - accuracy: 0.2000 - val_loss: 1.9069 - val_accuracy: 0.2030
Epoch 4/10
2561/2561 [==============================] - 5411s 2s/step - loss: 1.9039 - accuracy: 0.2079 - val_loss: 1.8649 - val_accuracy: 0.2279
Epoch 5/10
2561/2561 [==============================] - 5406s 2s/step - loss: 1.8574 - accuracy: 0.2211 - val_loss: 1.8172 - val_accuracy: 0.2340
Epoch 6/10
2561/2561 [==============================] - 5408s 2s/step - loss: 1.8093 - accuracy: 0.2350 - val_loss: 1.7792 - val_accuracy: 0.2594
Epoch 7/10
2561/2561 [==============================] - 5412s 2s/step - loss: 1.7710 - accuracy: 0.2487 - val_loss: 1.7377 -

In [43]:
# evaluate the network
print("[INFO] evaluating network...")
# create a generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(dataframe=df_test, directory=train_image_directory, x_col="imageFile", y_col=None, class_mode=None, target_size=(224, 224), batch_size=32, shuffle=False)

# use the model to make predictions
predictions = model.predict(test_generator)

# the predictions are probabilities for each class, so take the class with the highest probability as the prediction
predicted_classes = np.argmax(predictions, axis=1)

print("[INFO] evaluating network...")
test_loss, test_acc = model.evaluate(test_generator)
print("test loss: {:.2f}".format(test_loss))
print("test accuracy: {:.2f}".format(test_acc))

# save the model to disk
print("[INFO] serializing network...")
model.save("bttai-nybg-2024.model", save_format="smallVGNNET")

[INFO] evaluating network...
Found 0 validated image filenames.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 30690 invalid image filename(s) in x_col="imageFile". These filename(s) will be ignored.
  warnings.warn(


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
# plot the training loss and accuracy
import matplotlib.pyplot as plt
N = 10
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), model.history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), model.history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), model.history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), model.history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
